# IA para espacialização dos dados

In [35]:
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import rioxarray as rxr

from pyproj import Transformer
from rasterio.transform import rowcol

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, Dataset, DataLoader, random_split

### Device disponível para rodas a RNA

In [2]:
# 1) Dispositivo (GPU se disponível)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


## Dataset com os dados para convolução
---
- O Dataset nesta aplicação é extremamente importante pois irá englobar todas as informações necessárias para espacializar os dado de infiltração

In [84]:
# 2) Dataset
class MeuDataset(Dataset):
    nix_bands = [
        "R400 nm", "R410 nm", "R420 nm", "R430 nm",
        "R440 nm", "R450 nm", "R460 nm", "R470 nm",
        "R480 nm", "R490 nm", "R500 nm", "R510 nm",
        "R520 nm", "R530 nm", "R540 nm", "R550 nm",
        "R560 nm", "R570 nm", "R580 nm", "R590 nm",
        "R600 nm", "R610 nm", "R620 nm", "R630 nm",
        "R640 nm", "R650 nm", "R660 nm", "R670 nm",
        "R680 nm", "R690 nm", "R700 nm",
    ]

    columns_dados = ["Ponto", "Lat", "Lon", "Clay", "Silt", "Sand", "K (C1)"]

    def __init__(self, janela:int, device:torch.device|None=None):
        self.device = device
        self.janela = janela # Janela deve ser ímpar

        if self.janela%2 == 0:
            raise ValueError("A janela deve ser ímpar")

        # Lendo Tabelas
        print("Lendo Tabelas")
        self.dados = pd.read_excel(r"D:\Mestrado\Trabalho Final\Dados\Levantamento em Campo\Compiled.xlsx", sheet_name="Infiltracao")
        self.dados = self.dados[self.columns_dados].dropna().reset_index(drop=True)
        gdf = gpd.GeoDataFrame(self.dados, geometry=gpd.points_from_xy(self.dados["Lat"], self.dados["Lon"]), crs="EPSG:4326")
        gdf.to_crs("EPSG:31983", inplace=True)
        self.dados['Lat'] = gdf.geometry.y
        self.dados['Lon'] = gdf.geometry.x

        self.nix = pd.read_excel(r"D:\Mestrado\Trabalho Final\Dados\Levantamento em Campo\Compiled.xlsx", sheet_name="Nix")
        self.pXRF = pd.read_excel(r"D:\Mestrado\Trabalho Final\Dados\Levantamento em Campo\Compiled.xlsx", sheet_name="pXRF")

        # Lendo Rasteres importantes
        self.talvegues = gpd.read_file(r"D:/Mestrado/Trabalho Final/SIG/HidrografiaArea.zip")

        # Lendo Rasteres
        print("Lendo Rasteres")
        self.uso_solo        = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/USOSOLO.tif")                 # Tipos de uso do solo
        self.elevation       = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/Elevation.tif")               # Elevação
        self.terrain_rug_idx = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/TerrainRuggednessIndex.tif")  # Variação de elevação entre um pixel e seus vizinhos imediatos
        self.topo_pos_idx    = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/TopograficPositionIndex.tif") # Elevação de um ponto com a média da elevação ao redor, topo, vale ou plano
        self.roughness       = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/Roughness.tif")               # A diferença entre a elevação máxima e mínima dentro de uma vizinhança
        self.slope           = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/Slope.tif")                   # Declividade
        self.aspect          = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/Aspect.tif")                  # Para onde "aponta" a face do terreno

        # Dados dos rasteres concatenados
        self.raster_data = np.array([
            self.uso_solo.values[0],
            self.elevation.values[0],
            self.terrain_rug_idx.values[0],
            self.topo_pos_idx.values[0],
            self.roughness.values[0],
            self.slope.values[0],
            self.aspect.values[0],
        ])

        self.transformer = Transformer.from_crs("EPSG:31983", self.uso_solo.rio.crs, always_xy=True)
        self.transform = self.uso_solo.rio.transform()

        self._process_dados()

    def _process_dados(self):
        # Dados para convolução
        x, y = self.transformer.transform(self.dados["Lon"], self.dados["Lat"])
        row, col = rowcol(self.transform, x, y)

        jan = int((self.janela-1)/2)

        start_row = row-jan
        end_row   = row+jan

        start_col = col-jan
        end_col   = col+jan

        self.dados['s_row']=start_row
        self.dados['e_row']=end_row
        self.dados['s_col']=start_col
        self.dados['e_col']=end_col

        # Distância até o talvegue principal
        linhas_unidas = self.talvegues.union_all()
        pontos = gpd.GeoSeries(gpd.points_from_xy(self.dados["Lon"], self.dados["Lat"]), crs="EPSG:31983")
        dists = pontos.apply(lambda p: p.distance(linhas_unidas))
        self.dados["dist_talvegue"] = dists


    def __len__(self):
        return len(self.dados)

    def __getitem__(self, idx):
        if isinstance(idx, (int, float)):
            idx = [idx]

        # Pontos    
        pontos = self.dados.loc[idx]

        # Valores nos rasteres
        rasters_vals = []
        for s_row, e_row, s_col, e_col in zip(pontos['s_row'], pontos['e_row'], pontos['s_col'], pontos['e_col']):
            rasters_vals.append(self.raster_data[:, s_row:e_row+1, s_col:e_col+1])
        
        rasters_vals = torch.tensor(np.array(rasters_vals))

        # Demais dados
        K = torch.tensor(pontos["K (C1)"].values)
        dist_talvegue = torch.tensor(pontos["dist_talvegue"].values)

        return (rasters_vals, dist_talvegue), K
    
dataset = MeuDataset(janela = 25, device=device)

dataset[15][1], dataset[1:3][1]

Lendo Tabelas
Lendo Rasteres


(tensor([0.0057], dtype=torch.float64),
 tensor([0.0092, 0.0025, 0.0012], dtype=torch.float64))